<a href="https://colab.research.google.com/github/mdsl22/Coursera_Capstone/blob/master/Week_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 Introduction

This notebook will be used for the week 3 assignment of the Applied Data Science Capstone.

### 1.1 Download the required libraries

In [0]:
import numpy as np 
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2 Download and explore the dataset

### 2.1 Install Wikipedia module for Python

In [31]:
import sys
!{sys.executable} -m pip install wikipedia

### 2.2 Scrape data from Wikipedia into a csv file

In [0]:
import wikipedia as wp
 
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('toronto_data.csv',header=1,index=False)

### 2.3 Transform the data into a pandas dataframe

In [33]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df = pd.read_csv('toronto_data.csv')
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


In [34]:
#  Ignore cells with a borough that is Not assigned

df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [0]:
# If Neighborhood = Not assigned then Neighborhood = Borough

In [36]:
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])

df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [37]:
# Combine the rows of one postal code area

df_toronto = df.groupby('Postcode').agg(
    {
        'Borough':'first', 
        'Neighbourhood': ', '.join,}
    ).reset_index()

df_toronto.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [38]:
df_toronto.shape

(103, 3)

## 3 Latitude and the longitude coordinates of each neighborhood

In [39]:
# Obtain coordinates from the given csv file

!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-11-17 16:21:06--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-11-17 16:21:06--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-17 16:21:08--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-17 16:21:08--  https://ibm.box.com/public/static/9

In [40]:
# Read the csv data

df_cord = pd.read_csv('GeoCord.csv') 
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
# Create Latitude and Longitude columns in the dataframe df_toronto
df_toronto['Latitude'] = np.nan
df_toronto['Longitude'] = np.nan

In [51]:
# For each postcode in df_toronto find the corresponding coordinates in df_cord and assign it to df_toronto
for idx in df_toronto.index:
    cord_idx = df_cord['Postal Code'] == df_toronto.loc[idx, 'Postcode']
    df_toronto.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_toronto.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

df_toronto.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## 4 Explore and cluster the neighborhoods in Toronto

In [52]:
# Geographical coordinates of Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [58]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitude, longitude, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### 4.1 Foursquare credentials and version

In [61]:
CLIENT_ID = 'S52NPP2NCWKLHPGXFJLLGLUJEROEI32DGTYBGUDOM0XKLYGB' # your Foursquare ID
CLIENT_SECRET = 'KVF0LGIYW1FNVD1WHD30WC1EGWMMX5ISCAACN2YJTTBVM2CJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S52NPP2NCWKLHPGXFJLLGLUJEROEI32DGTYBGUDOM0XKLYGB
CLIENT_SECRET:KVF0LGIYW1FNVD1WHD30WC1EGWMMX5ISCAACN2YJTTBVM2CJ
